In [2]:
import googlemaps
import pandas as pd
import pickle

In [2]:
# keyfile = '/dartfs/rc/lab/P/Psych81.09/course/shared/gmaps.api.key'
# set keyfile path, open, and read private API key
# keyfile = 'gmaps.api.key'
# fd = open(keyfile, 'r')
# key = fd.readline()[:-1]
# fd.close()

In [3]:
# pass key to google maps client for auth
gmaps = googlemaps.Client(key=key)

In [8]:
# read in data
df=pd.read_csv('parking-violations-geodata.csv', delimiter=',', encoding = 'latin-1', index_col='Unnamed: 0', 
               dtype={'Zip Code' : int})

In [5]:
# create dictionary to hold geodata. {row number : geodata}
geodata = dict()

for ix, row in df.iterrows():
    
    # format address for google maps API call
    fmt_addr = str(row['House Number']) + ' ' + row['Street Name'] + ', ' + row['City & State']
    
    # not sure how google maps deals with errors, so handle a few possibilities
    try:
        # commented out to avoid accidental calls
#         addr_info = gmaps.geocode(fmt_addr)

    # identify rows where API throws error, but don't stop loop
    except:
        print(f'Error in row {ix}')
        continue
        
    # maybe API returns an empty list when no matching address is found?
    if addr_info:
        geodata[ix] = addr_info
    else:
        print(f'No data returned for row {ix}')
        

In [8]:
# save out dictionary with geodata
# with open('geodata.p', 'wb') as f:
#     pickle.dump(geodata, f)

In [50]:
# add geodata to violations dataframe
for ix, row in df.iterrows():
    # index (insanely organized) google maps data object
    for component in geodata[ix][0]['address_components']:
        # locate zip code and add to each row
        if 'postal_code' in component['types']:   
            df.at[ix, 'Zip Code'] = int(component['short_name'])
     
    # then add latitude and longitude coordinates
    df.at[ix, 'Lat'] = geodata[ix][0]['geometry']['location']['lat']
    df.at[ix, 'Long'] = geodata[ix][0]['geometry']['location']['lng']

In [56]:
# save out dataframe with geodata!
# df.to_csv('parking-violations-geodata.csv')

In [11]:
# example of google maps API data format
geodata[4]

[{'address_components': [{'long_name': '133',
    'short_name': '133',
    'types': ['street_number']},
   {'long_name': 'Frost Street', 'short_name': 'Frost St', 'types': ['route']},
   {'long_name': 'Williamsburg',
    'short_name': 'Williamsburg',
    'types': ['neighborhood', 'political']},
   {'long_name': 'Brooklyn',
    'short_name': 'Brooklyn',
    'types': ['political', 'sublocality', 'sublocality_level_1']},
   {'long_name': 'Kings County',
    'short_name': 'Kings County',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'New York',
    'short_name': 'NY',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']},
   {'long_name': '11211', 'short_name': '11211', 'types': ['postal_code']}],
  'formatted_address': '133 Frost St, Brooklyn, NY 11211, USA',
  'geometry': {'bounds': {'northeast': {'lat': 40.7181074, 'lng': -73.9462073},
    'southwest': {'

In [9]:
# check that geodata added correctly
df.head(5)

,Summons Number,Violation Code,Violation Time,House Number,Street Name,City & State,Zip Code,Lat,Long
0,8725653684,71,0319P,235,Bedford Ave,"Brooklyn, NY",11211,40.715957,-73.959527
1,8725653696,20,0322P,235,Bedford Ave,"Brooklyn, NY",11211,40.715957,-73.959527
2,8725653702,20,0324P,235,Bedford Ave,"Brooklyn, NY",11211,40.715957,-73.959527
3,8670481194,21,1138A,133,Frost St,"Brooklyn, NY",11211,40.718053,-73.946250
4,8670481212,21,1140A,133,Frost St,"Brooklyn, NY",11211,40.718053,-73.946250
